In [1]:
# -*- coding: utf-8 -*-
"""

@author: 
"""


import gzip,_pickle,random
import numpy as np
import logging
import time

In [2]:
# Creating log file

logger1=logging.getLogger(__name__)

logger1.setLevel(logging.DEBUG)

formatter=logging.Formatter('%(message)s')
file_handler=logging.FileHandler('C:\\Users\\Welcome\\Desktop\\Code files(IMPORTANT)\\Codes\\Neural Networks\\Neural Network(MNIST)\\NN log files\\NNpatch(Mine).log', 'w')
file_handler.setFormatter(formatter)
logger1.addHandler(file_handler)

In [3]:
# Data loading and preprocessing

def load_data():
    with gzip.open('C:/Users/Welcome/Downloads/mnist.pkl.gz','rb') as file:
        training_data,validation_data,test_data=_pickle.load(file,encoding='latin1')

    training_inputs = [np.reshape(x, (784, 1)) for x in training_data[0]]
    training_results = [vectorized_results(y) for y in training_data[1]]
    training_data = list(zip(training_inputs, training_results))

    validation_inputs = [np.reshape(x, (784, 1)) for x in validation_data[0]]
    validation_results = validation_data[1]
    validation_data = list(zip(validation_inputs, validation_results))

    test_inputs = [np.reshape(x, (784, 1)) for x in test_data[0]]
    test_data = list(zip(test_inputs, test_data[1]))
    
    #############################################################3
    training_data=[training_data[tr] for tr in range(int(len(training_data)/100))]
    validation_data=[validation_data[v] for v in range(int(len(validation_data)/100))]
    test_data=[test_data[te] for te in range(int(len(test_data)/100))]
    
    return training_data, validation_data, test_data


def vectorized_results(label):
    e=np.zeros((10,1))
    e[label]=1.0
    return e

trainingData, validationData, testData=load_data()


In [4]:
# Network class

class Network(object):

    def __init__(self, sizes, learning_rate=1.0,mini_batch_size=16, epochs=10):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.eta=learning_rate
        self.mini_batch_size=mini_batch_size
        self.epochs=epochs
        np.random.seed(0)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        np.random.seed(0)
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        #self.biases = [np.random.randint(0,high=10,size=(y, 1)) for y in sizes[1:]]
        #self.weights = [np.random.randint(0,high=10,size=(y, x))/np.sqrt(x)
         #                 for x, y in zip(sizes[:-1], sizes[1:])]
    
    def feedforward(self, a):
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    #net.SGD(trainingData, 3, 2, 3.0, test_data=testData)
    def fit(self, trainingData, test_data=None):
        start=time.time()
        logger1.debug('######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                     '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '###################################          Start     #############################################################\n'
                      '###################################                    ###############################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n'
                      '######################################################################################################################\n')
        if test_data: n_test = len(test_data)
        n = len(trainingData)
        for epoch in range(1,self.epochs+1):
            logger1.debug('\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>EPOCH {} <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(epoch))
            ###################################
            random.seed(0)
            random.shuffle(trainingData)
            mini_batches = [
                trainingData[k:k+self.mini_batch_size]
                for k in range(0, n, self.mini_batch_size)]
            mini=1
            for mini_batch in mini_batches:
                logger1.debug('\n-------------------------------------------------Mini_batch {} ------------------'.format(mini))
                
                #Following if/else is to index Training Inputs
                
                if mini<2:
                    No=mini
                else:
                    No=(mini*2)-1
                mini+=1
                #############
                self.update_mini_batch(mini_batch,No)
            if test_data:
                logger1.debug("My Epoch {}: {} / {}".format(
                    epoch, self.evaluate(test_data), n_test))
                print("My Epoch {}: {} / {}".format(
                    epoch, self.evaluate(test_data), n_test))
            else:
                logger1.debug("My Epoch {} complete".format(epoch))
                print("My Epoch {} complete".format(epoch))
        logger1.debug('{}'.format(time.time()-start))

    def update_mini_batch(self, mini_batch,nth_input):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            logger1.debug('\n--------------------------------------------------- Training Input {}'.format(nth_input))
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nth_input+=1######
        logger1.debug('\n                                               Gradient descent/Updating Weights and Biases') 
        self.weights = [w-(self.eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(self.eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x] 
        zs = [] 
        logger1.debug('\n                          ---------------------------Feedforward')
        logger1.debug('\n\nInput activations\n{} \n\nShape of input activations {}\n\n'.format(activation.reshape(28,28),activation.shape))
        for b, w in zip(self.biases, self.weights):
            #dotwa=np.dot(w, activation)
            
            dotwa=w.dot(activation)
            
            
            #TAKE DIAGONAL FOR WEIGHTED INPUTS**********
            #*******************************************
            #z = (dotwa+b).diagonal()#TAKE DIAGONAL FOR WEIGHTED INPUTS
            #*******************************************
            #TAKE DIAGONAL OF WEIGHTED INPUT MATRIX FOR 
            #CALCULATING WEIGHTED INPUT VALUES**********
            
            z = (dotwa+b)
            
            zs.append(z)
            activation = sigmoid(z)
            
            logger1.debug('\n\nThese are Weights\n {}\nShape of '
                  'weight{}\n\n'
                  'These are biases\n{}\nShape of biase{}\n\nWeights'
                  'dotted with activation\n{}\nShape of Weights dotted with activation{}\n\nThese are weighted'
                  ' inputs\n{}\nShape of weighted inputs{}\n\nThese '
                  'are activations\n{}\nShape of '
                  'activation{}'.format(w,w.shape,b,b.shape,dotwa,dotwa.shape,z,z.shape,activation,activation.shape))
            
            activations.append(activation)
             
        ########self.cost_derivative(activations[-1], y)).diagonal()#######
        #predelta = (self.cost_derivative(activations[-1], y)).diagonal() * \
        #    sigmoid_prime(zs[-1])#Took predelta's shape's first element
        #predelta1st=(predelta.shape)[0]#Then reshaped delta from(10,) to (10,1)
        #delta=predelta.reshape(predelta1st,1)
        
        
        delta=(self.cost_derivative(activations[-1], y)) * sigmoid_prime(zs[-1]) 
        
        ###################################################################
        nabla_b[-1] = delta
        
        logger1.debug('\n\nyvalues /Actual output values/Training labels\n{}Shape of y'
              '{}\n\nself.cost_derivative(activations[-1], y)\n{}'
              '\nShape of self.cost_derivative(activations[-1], y)'
              '{}\n\nSigmoid_prime(zs[-1])\n{}\nShape of sigmoid_prime(zs[-1])'
              '{}\n\nError(Delta)/nabla_b[-1] values\n{}\nShape of '
              'nabla_b[-1]{}\n\nLast second'
              ' layer activations\n{}\nLast second'
              ' layer activations transpose\n{}\n'.format(y,
                                             y.shape,
                                             (self.cost_derivative(activations[-1], y)),
                                             (self.cost_derivative(activations[-1], y)).shape,
                                             sigmoid_prime(zs[-1]),
                                             sigmoid_prime(zs[-1]).shape,
                                             delta,
                                             delta.shape,
                                             activations[-2],activations[-2].transpose()))
        
       
        '''logger1.debug('\n\nyvalues /Actual output values/Training labels\n{}Shape of y'
              '{}\n\nself.cost_derivative(activations[-1], y).diagonal()\n{}'
              '\nShape of self.cost_derivative(activations[-1], y).diagonal()'
              '{}\n\nSigmoid_prime(zs[-1])\n{}\nShape of sigmoid_prime(zs[-1])'
              '{}\n\nError(Delta)/nabla_b[-1] values\n{}\nShape of '
              'nabla_b[-1]{}\n\nLast second'
              ' layer activations\n{}\n\nLast second'
              ' layer activations transpose\n{}'.format(y,
                                             y.shape,
                                             (self.cost_derivative(activations[-1], y)).diagonal(),
                                             (self.cost_derivative(activations[-1], y)).diagonal().shape,
                                             sigmoid_prime(zs[-1]),
                                             sigmoid_prime(zs[-1]).shape,
                                             delta,
                                             delta.shape,
                                             activations[-2],
                                             (activations[-2].transpose()).reshape(1,((activations[-2].transpose()).shape)[0])))'''
             
      
              
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        #nabla_w[-1] = np.dot(delta, (activations[-2].transpose())\
         #      .reshape(1,((activations[-2].transpose()).shape)[0]))
        #Reshaped activations[-2].transpose() from shape(3,) to (1,3)
        ###################################################################
        
        logger1.debug('\n\nnabla_w[-1] values\n{}\nShape of '
              'nabla_w[-1] values{}'.format(nabla_w[-1],nabla_w[-1].shape))
              
        
        for l in range(2, self.num_layers):
            logger1.debug('\n                       --------------------------Backpropagation')
            z = zs[-l]
            sp = sigmoid_prime(z)
            #########################################
            self_weights=self.weights[-l+1]
            self_weightstranspose=self_weights.transpose()
            self_weightsdotDelta=np.dot(self_weightstranspose, delta)
            delta = (self_weightsdotDelta * sp)
            #############################################
            nabla_b[-l] = delta
            
            logger1.debug('\n\nz/zs[-l] values\n{}\nz/zs[-l] values shape'
                  '{}\n\nsigmoid_prime(z)\n{}\n{}'
                  '\n\nself.weights[-l+1]\n{}'
                  '\n{}\n\nself.weights[-l+1].transpose()\n{}\n{}'
                  '\n\nnp.dot(self.weights[-l+1].transpose(), delta)\n{}shape{}'
                  '\n\nThis is Error(Delta)\n{}Shape of '
                  'Error(Delta){}\n\n\nShape of '
                  'Backwardpass Activations activations[-l-1]{}\n\nShape of '
                  'Transposed Backwardpass activations[-l-1].transpose()'
                  '{}\n\n'.format(z,z.shape,sp,sp.shape,self_weights,
                   self_weights.shape,self_weightstranspose,
                   self_weightstranspose.shape,self_weightsdotDelta,
                   self_weightsdotDelta.shape,delta,delta.shape,
                   activations[-l-1].shape,activations[-l-1].transpose().shape))
                       
            
            ##########################################################3
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            ###########################################################
        return (nabla_b, nabla_w)

    def evaluate(self, testData):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in testData]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)


In [5]:
#### Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [6]:
# Implementing the network

net=Network([784,3,10], learning_rate=1.0, mini_batch_size=2, epochs=5)
net.fit(trainingData,test_data=testData)

My Epoch 1: 11 / 100
My Epoch 2: 18 / 100
My Epoch 3: 20 / 100
My Epoch 4: 19 / 100
My Epoch 5: 34 / 100
